In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

In [ ]:
# Location of training file for our training dataset
train_file = "../input/landmark-recognition-2021/train.csv"

In [ ]:
# Reading file using pandas "read_csv" 
data = pd.read_csv(train_file)

In [ ]:
# Exploring the dataset
data.head()

In [ ]:
# Exploring the target values and that is one the crucial task need to be taken care of

print(data["landmark_id"].max(), data["landmark_id"].min(), data["landmark_id"].nunique())

# seems like our data containes label inconsistent so we need a mapping dictiory for mapping our target values associated with 
#our input_image

reverse_target_mapping = { i : value for i,value in enumerate(sorted(data["landmark_id"].unique()))}

# Here we are exchaning the keys and values in each other for mapping purpose
target_mapping = {value : key for key,value in zip(reverse_target_mapping.keys(), reverse_target_mapping.values())}

target_mapping

In [ ]:
# Colleting our input image file location using tf.io.gfile.glob, for more info look at official documentation


files = tf.io.gfile.glob("../input/landmark-recognition-2021/train/*/*/*/*.jpg")

In [ ]:
# Checking the files consistency with our input train file whether any value is missing or not
len(files)

In [ ]:
# Sorting the value for concatnating our input image train file for processing
data.sort_values(inplace = True, by = "id")

In [ ]:
# Reseting the indices

data.reset_index(inplace = True)

In [ ]:
# droping the previous indices column , this can be done using reset index = True, but i forgot
data.drop(columns = ["index"] , inplace = True)

In [ ]:
# As we can see in the output , our landmark id contains inconsistent label values so we need to reverse map with
# our target dictionary
data.head()

In [ ]:
data["landmark_id"] = data["landmark_id"].map(target_mapping)

# we can double check for target mapping
print(target_mapping[107382])

data.head()

In [ ]:
# Saving processed this input_file into input directory
os.makedirs("./precessed")
data.to_csv("./precessed/training_file_processed.csv")

series = pd.DataFrame(data = {"true_label" : target_mapping.keys(), "mapped_label" : target_mapping.values()})

series.to_csv("./precessed/label_mapping.csv")

In [ ]:
data["input_file_loc"] = sorted(files)

In [ ]:
data.head()

In [ ]:
data["id"].loc[0], data["input_file_loc"].loc[0]

# Final Dataset Making

In [ ]:
# Dataset making 


file_dataset = tf.data.Dataset.from_tensor_slices(data["input_file_loc"].values)
target_dataset = tf.data.Dataset.from_tensor_slices(data["landmark_id"].values)



In [ ]:

next(iter(file_dataset)), next(iter(target_dataset))

# Processing function

In [ ]:
# Normalizing the images to [-1, 1]
def normalize(img):
    img = (img / 127) - 1
    return img

def prepare_dataset(image):
    img = tf.io.read_file(image)
    img = tf.image.decode_jpeg(img , channels = 3)
    img = tf.image.resize(img, size = (299, 299))
    
    return img



In [ ]:
file_dataset = file_dataset.map(prepare_dataset, num_parallel_calls = tf.data.AUTOTUNE)
file_dataset = file_dataset.map(normalize, num_parallel_calls = tf.data.AUTOTUNE)

In [ ]:
next(iter(file_dataset)), next(iter(target_dataset))

In [ ]:
final_dataset = tf.data.Dataset.zip((file_dataset, target_dataset)).batch(32, drop_remainder = True)

In [ ]:
# Model Building
next(iter(final_dataset))[0].shape, next(iter(final_dataset))[1].shape

# Model building

In [ ]:
model = tf.keras.applications.MobileNetV3Large(include_top = True,weights = None ,input_shape = (299,299,3), classes = 81313)
model.summary()

# Model Compilation

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics = ["accuracy"])

# Model Fitting and Evaluation

In [ ]:
model.fit(final_dataset, epochs = 1, workers = 4, use_multiprocessing = True)